In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys

In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
from PIL import Image

In [3]:
from datasets import RoadsDatasetTrain, RoadsDatasetTest
from model import UNet
from train import train
from predict import predict

In [4]:
BATCH_SIZE = 1
EPOCHS = 1
LEARNING_RATE = 0.0001

PATCH_SIZE = 16
LARGE_PATCH_SIZE = 96

TRAIN_IMAGE_INITIAL_SIZE = 400
NUMBER_PATCH_PER_TRAIN_IMAGE = (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TRAIN_IMAGE_INITIAL_SIZE // PATCH_SIZE)

TEST_IMAGE_INITIAL_SIZE = 608
NUMBER_PATCH_PER_TEST_IMAGE = (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE) * (TEST_IMAGE_INITIAL_SIZE // PATCH_SIZE)
CRITERION = nn.BCELoss()

In [5]:
train_data_dir = "./Datasets/training"
train_dataset = RoadsDatasetTrain(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TRAIN_IMAGE,image_initial_size= TRAIN_IMAGE_INITIAL_SIZE, root_dir=train_data_dir)
train_dataloader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

test_data_dir = "./Datasets/test_set_images"
test_dataset = RoadsDatasetTest(patch_size=PATCH_SIZE, large_patch_size=LARGE_PATCH_SIZE, number_patch_per_image=NUMBER_PATCH_PER_TEST_IMAGE,image_initial_size= TEST_IMAGE_INITIAL_SIZE,root_dir=test_data_dir)
test_dataloader = data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [6]:
# unet = UNet()
# train(model=unet, dataloader=train_dataloader, epochs=EPOCHS, criterion=CRITERION)

In [7]:
unet = torch.load('./Model_2', map_location=torch.device('cpu'))

In [8]:
predict(unet, test_dataloader)

[Patch 0/2888]
[Patch 100/2888]
[Patch 200/2888]
[Patch 300/2888]
[Patch 400/2888]
[Patch 500/2888]
[Patch 600/2888]
[Patch 700/2888]
[Patch 800/2888]
[Patch 900/2888]
[Patch 1000/2888]
[Patch 1100/2888]
[Patch 1200/2888]
[Patch 1300/2888]
[Patch 1400/2888]
[Patch 1500/2888]
[Patch 1600/2888]
[Patch 1700/2888]
[Patch 1800/2888]
[Patch 1900/2888]
[Patch 2000/2888]
[Patch 2100/2888]
[Patch 2200/2888]
[Patch 2300/2888]
[Patch 2400/2888]
[Patch 2500/2888]
[Patch 2600/2888]
[Patch 2700/2888]
[Patch 2800/2888]


In [9]:
# result = torch.zeros(len(test_dataloader),3, PATCH_SIZE,PATCH_SIZE)
# indices = np.zeros((len(test_dataloader), 3), dtype=int)
# PADDING = 40

# def crop(image):
#     return image[:,PADDING:PADDING+PATCH_SIZE, PADDING:PADDING+PATCH_SIZE]

# l=None
# for i, k in enumerate(test_dataloader):
#         k['image'][0]
#         if(i==1):
#             l=k['image'][0]
#         result[i] = crop(k['image'][0])
#         indices[i] = np.array([k['id'], k['x'], k['y']])
        

# def reconstruct_images(result, indices):
#     images = torch.zeros((indices.shape[0]//1444,3, 608, 608))
    
#     for i in range(indices.shape[0]):
#         image_number = indices[i, 0]
        
#         start_x = indices[i, 1] * PATCH_SIZE
#         end_x = start_x + PATCH_SIZE
        
#         start_y = indices[i, 2] * PATCH_SIZE
#         end_y = start_y + PATCH_SIZE 
        
#         images[image_number,:, start_x:end_x, start_y:end_y] = result[i]
        
#     return images

# reconstructed = reconstruct_images(result, indices)
# mask = reconstructed.clone().detach().type(torch.float)



# # im = transforms.ToPILImage()(torch.tensor(result[0], dtype=torch.float)).convert('RGB')
# # im
# im = transforms.ToPILImage()(mask[1]).convert('RGB')
# im